In [1]:
import os

In [2]:
os.chdir("../")

In [5]:
from sklearn.metrics import silhouette_score
import dagshub
import mlflow
from urllib.parse import urlparse
from pathlib import Path
from src.Customer_segementation.utils.common import load_object

In [6]:
import pandas as pd

In [12]:
class ModelEvaluation:
    def __init__(self) -> None:
        pass

    def evluation_matrix_for_mlflow(self, model, train_data): 
        dagshub.init(repo_owner='Abhikkumar619', repo_name='Customer_segmentation_project', mlflow=True)
        mlflow.set_registry_uri('https://dagshub.com/Abhikkumar619/Customer_segmentation_project.mlflow')
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)


        with mlflow.start_run():
            prediction=model.predict(train_data)
            sil_score=silhouette_score(train_data ,model.labels_)
            mlflow.log_metric("silhouette_score",sil_score)

            if tracking_url_type_store !='file': 
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ml_model")
            else: 
                mlflow.sklearn.log_model(model, "model")


    def inititate_model_evaluation(self):
        try: 
            # Load model from artifacts.
            model=load_object(Path("artifacts/model_trainer/best_Model.h5"))
            dim_model=load_object(Path("artifacts/model_trainer/dim_red.h5"))
            train_data_path="artifacts/data_transformation/train_scaled.csv"
            test_data_path="artifacts/data_transformation/test_scaled.csv"

            x=pd.read_csv(train_data_path)
            y=pd.read_csv(test_data_path)

            dim_train=dim_model.transform(x)

            self.evluation_matrix_for_mlflow(model, dim_train)

           
        except Exception as e: 
            raise



In [13]:
try: 
    model_evaluation=ModelEvaluation()
    model_evaluation.inititate_model_evaluation()
    
except Exception as e:
    raise e

[2024-07-01 06:35:57,137 : INFO : common : object load sucessfully from path: artifacts/model_trainer/best_Model.h5]
[2024-07-01 06:35:57,139 : INFO : common : object load sucessfully from path: artifacts/model_trainer/dim_red.h5]


Initialized MLflow to track repo "Abhikkumar619/Customer_segmentation_project"

[2024-07-01 06:35:57,690 : INFO : helpers : Initialized MLflow to track repo "Abhikkumar619/Customer_segmentation_project"]


Repository Abhikkumar619/Customer_segmentation_project initialized!

[2024-07-01 06:35:57,692 : INFO : helpers : Repository Abhikkumar619/Customer_segmentation_project initialized!]
https


/Users/abishekkumaryadav/DataScience/Machine_learning/Customer_segmentation_project/cus_env/lib/python3.8/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ml_model'.
2024/07/01 06:36:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ml_model, version 1
Created version '1' of model 'ml_model'.
